# Feature Selection

### Load the data from file and save train/test data

In [2]:
#code
import numpy as np
import pandas as pd

df = pd.read_csv('tmp/extracted_data/extracted_data.csv')
df.dropna(axis=1, inplace=True)
df.replace([np.inf, -np.inf], np.nan).dropna(axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)

df.head()

,Flow rate__abs_energy,Flow rate__absolute_sum_of_changes,"Flow rate__agg_autocorrelation__f_agg_""mean""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""median""__maxlag_40","Flow rate__agg_autocorrelation__f_agg_""var""__maxlag_40","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,Zone29_K1,Zone29_K2,Zone29_K3,Zone30_K1,Zone30_K2,Zone30_K3,Zone31_K1,Zone31_K2,Zone31_K3,target
0,8.439290e-08,0.000074,0.0,0.0,0.0,0.000013,-0.480982,-9.571038e-08,1.838974e-08,0.000014,...,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
1,8.337982e-08,0.000071,0.0,0.0,0.0,0.000013,-0.616363,-1.284958e-07,1.649928e-08,0.000015,...,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
2,1.032448e-07,0.000059,0.0,0.0,0.0,0.000012,-0.241002,-4.498370e-08,1.976525e-08,0.000014,...,4.000000e-11,5.000000e-11,2.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,0
3,9.590627e-08,0.000061,0.0,0.0,0.0,0.000013,-0.528130,-1.032466e-07,1.703194e-08,0.000015,...,4.000000e-11,5.000000e-11,2.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,4.000000e-11,5.000000e-11,2.000000e-12,1
4,9.772771e-08,0.000060,0.0,0.0,0.0,0.000013,-0.493841,-9.472176e-08,1.738857e-08,0.000014,...,8.000000e-11,9.000000e-11,3.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,8.000000e-11,9.000000e-11,3.000000e-12,1


In [3]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [4]:
y.value_counts()

1    5055
0    4959
Name: target, dtype: int64

### Automatic sccoring function

In [5]:
#code
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score, f1_score

seed = 0

def automatic_scoring_cv(X, y):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=seed)
    average_score = cross_val_score(estimator=rf_model, X=X, y=y, cv=5, scoring='roc_auc', n_jobs=-1).mean()
    return 'cv + roc_auc: ' + str(average_score)

def automatic_scoring_tt(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed, stratify=y)
    rf_model = RandomForestClassifier(n_estimators=100, random_state=seed).fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    return 'tt + f1_score: ' + str(f1_score(y_test, y_pred))

In [6]:
automatic_scoring_cv(X, y)

'cv + roc_auc: 0.8981465997252126'

In [7]:
automatic_scoring_tt(X, y)

'tt + f1_score: 0.8166816681668169'

### 1. Feature selector that removes all low-variance features.

In [8]:
#code
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=(.8 * (1 - .8)))
selector.fit(X)
columns = X.columns[selector.get_support(indices=True)]
X = X[columns]

X.head()

,Flow rate__ar_coefficient__k_10__coeff_1,Flow rate__ar_coefficient__k_10__coeff_2,Flow rate__ar_coefficient__k_10__coeff_3,Flow rate__ar_coefficient__k_10__coeff_4,"Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_aggregated__aggtype_""kurtosis""",...,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__sum_of_reoccurring_values,Zone9_Pressure__sum_values,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_1,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_2,Zone9_Pressure__time_reversal_asymmetry_statistic__lag_3,Zone9_Pressure__value_count__value_0,Zone9_Pressure__variance
0,1.097224,-0.119454,-0.018179,0.013075,1.0,2.979672,372.0,544.0,49.084752,8.469481,...,0.0,0.0,5.125808e+06,0.0,1.0,2.200324e+11,4.326221e+11,6.275744e+11,749.0,1.887843e+08
1,1.157133,-0.161022,-0.005301,-0.040777,1.0,2.496999,380.0,629.0,51.389448,9.124820,...,0.0,0.0,1.736141e+07,0.0,0.0,9.831942e+11,1.959747e+12,2.929644e+12,752.0,1.132615e+09
2,1.076403,-0.057993,-0.032339,-0.005114,1.0,2.028811,425.0,435.0,32.345881,12.249977,...,0.0,0.0,4.831920e+06,1.0,1.0,4.153841e+11,8.162606e+11,1.207333e+12,706.0,2.478467e+08
3,1.083679,-0.072918,-0.031834,-0.004072,1.0,2.406142,425.0,538.0,46.956311,9.386179,...,0.0,0.0,1.171068e+07,0.0,0.0,4.574290e+11,9.116276e+11,1.364312e+12,700.0,5.662582e+08
4,1.050232,-0.047799,-0.043900,0.013567,1.0,2.456812,410.0,525.0,45.288313,9.376401,...,0.0,0.0,6.048610e+06,0.0,1.0,2.854228e+11,5.622315e+11,8.364422e+11,689.0,2.198799e+08


In [9]:
automatic_scoring_cv(X, y)

'cv + roc_auc: 0.8934594566575715'

In [10]:
automatic_scoring_tt(X, y)

'tt + f1_score: 0.810924369747899'

### 2. Feature selector that checks correlation between 2 features. If correlation is high, 1 feature is removed.

In [11]:
#code
import numpy as np

def correl(data, threshold):
    corr = data.corr()
    print('Corr calculated')
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= threshold:
                if columns[j]:
                    columns[j] = False
    selected_columns = data.columns[columns]
    data = data[selected_columns]
    return data

X = correl(X, 0.9)

X.head()

Corr calculated


,Flow rate__ar_coefficient__k_10__coeff_1,Flow rate__ar_coefficient__k_10__coeff_4,"Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_aggregated__aggtype_""kurtosis""","Flow rate__fft_coefficient__coeff_10__attr_""angle""","Flow rate__fft_coefficient__coeff_11__attr_""angle""",...,Zone9_Pressure__partial_autocorrelation__lag_9,Zone9_Pressure__quantile__q_0.7,Zone9_Pressure__range_count__max_1__min_-1,Zone9_Pressure__skewness,Zone9_Pressure__spkt_welch_density__coeff_2,Zone9_Pressure__spkt_welch_density__coeff_5,Zone9_Pressure__spkt_welch_density__coeff_8,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002
0,1.097224,0.013075,1.0,2.979672,372.0,544.0,49.084752,8.469481,2.951076,168.159905,...,-0.002299,0.0,749.0,2.331234,9.414505e+08,3.132578e+07,1.495198e+07,0.0,0.0,1.0
1,1.157133,-0.040777,1.0,2.496999,380.0,629.0,51.389448,9.124820,-105.444874,177.279461,...,-0.015294,0.0,752.0,1.532461,5.196042e+09,2.919493e+08,3.334011e+07,0.0,0.0,0.0
2,1.076403,-0.005114,1.0,2.028811,425.0,435.0,32.345881,12.249977,65.125545,-55.917921,...,-0.026681,0.0,706.0,2.915235,5.852888e+06,3.769895e+05,4.676094e+04,0.0,1.0,1.0
3,1.083679,-0.004072,1.0,2.406142,425.0,538.0,46.956311,9.386179,136.341265,-148.918332,...,-0.015482,0.0,700.0,1.604860,3.360284e+09,8.031815e+07,2.651042e+07,0.0,0.0,0.0
4,1.050232,0.013567,1.0,2.456812,410.0,525.0,45.288313,9.376401,-161.838119,122.674278,...,-0.010105,0.0,689.0,2.391474,1.057012e+08,1.403607e+07,6.431597e+06,0.0,0.0,1.0


In [12]:
automatic_scoring_cv(X, y)

'cv + roc_auc: 0.8811538224504571'

In [13]:
automatic_scoring_tt(X, y)

'tt + f1_score: 0.8023529411764705'

In [14]:
X['target'] = y
X.to_csv('tmp/selected_data_1.csv')

### 3. Select features according to a percentile of the highest scores (mutual_info_classif)

In [24]:
import numpy as np
import pandas as pd

df = pd.read_csv('tmp/selected_data_1.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

df.head()

,Flow rate__ar_coefficient__k_10__coeff_1,Flow rate__ar_coefficient__k_10__coeff_4,"Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_aggregated__aggtype_""kurtosis""","Flow rate__fft_coefficient__coeff_10__attr_""angle""","Flow rate__fft_coefficient__coeff_11__attr_""angle""",...,Zone9_Pressure__quantile__q_0.7,Zone9_Pressure__range_count__max_1__min_-1,Zone9_Pressure__skewness,Zone9_Pressure__spkt_welch_density__coeff_2,Zone9_Pressure__spkt_welch_density__coeff_5,Zone9_Pressure__spkt_welch_density__coeff_8,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002,target
0,1.097224,0.013075,1.0,2.979672,372.0,544.0,49.084752,8.469481,2.951076,168.159905,...,0.0,749.0,2.331234,9.414505e+08,3.132578e+07,1.495198e+07,0.0,0.0,1.0,0
1,1.157133,-0.040777,1.0,2.496999,380.0,629.0,51.389448,9.124820,-105.444874,177.279461,...,0.0,752.0,1.532461,5.196042e+09,2.919493e+08,3.334011e+07,0.0,0.0,0.0,0
2,1.076403,-0.005114,1.0,2.028811,425.0,435.0,32.345881,12.249977,65.125545,-55.917921,...,0.0,706.0,2.915235,5.852888e+06,3.769895e+05,4.676094e+04,0.0,1.0,1.0,0
3,1.083679,-0.004072,1.0,2.406142,425.0,538.0,46.956311,9.386179,136.341265,-148.918332,...,0.0,700.0,1.604860,3.360284e+09,8.031815e+07,2.651042e+07,0.0,0.0,0.0,1
4,1.050232,0.013567,1.0,2.456812,410.0,525.0,45.288313,9.376401,-161.838119,122.674278,...,0.0,689.0,2.391474,1.057012e+08,1.403607e+07,6.431597e+06,0.0,0.0,1.0,1


In [25]:
from sklearn.feature_selection import SelectPercentile, mutual_info_classif

selperc = SelectPercentile(mutual_info_classif, percentile=1)
X_features = selperc.fit_transform(X, y)
columns = np.asarray(X.columns.values)
support = np.asarray(selperc.get_support())
columns_with_support = columns[support]

X = X[columns_with_support]
X.head()

,"Flow rate__fft_aggregated__aggtype_""centroid""","Flow rate__fft_coefficient__coeff_1__attr_""angle""","Zone10_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone11_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","Zone11_Pressure__fft_aggregated__aggtype_""centroid""","Zone14_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone15_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone15_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","Zone15_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",Zone15_Pressure__count_below_mean,...,"Zone31_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone31_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone3_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone4_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone6_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","Zone6_Pressure__fft_coefficient__coeff_2__attr_""angle""",Zone6_Pressure__partial_autocorrelation__lag_2,"Zone7_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone7_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone7_Pressure__fft_coefficient__coeff_3__attr_""angle"""
0,49.084752,-120.106950,-7.922041e+04,148.990311,50.917722,103.922087,0.414992,41.544038,41.544038,637.0,...,90.663463,-6.422572e+04,110.365035,110.278247,156.500929,114.538620,-0.269811,0.465443,92.618935,139.140951
1,51.389448,-110.924886,8.590550e+05,159.261794,61.470542,99.293878,3.125592,42.191759,42.191759,670.0,...,71.662321,1.050872e+06,95.690130,95.042478,170.289578,101.596777,-0.043765,2.991166,71.918399,47.388674
2,32.345881,-156.802057,-1.078723e+06,60.594163,44.667697,105.809840,-1.049866,135.024234,135.024234,579.0,...,97.961497,-8.241194e+05,125.073052,127.395796,131.877713,121.304061,-0.269597,-1.148494,86.354413,127.533329
3,46.956311,-136.572893,3.193206e+05,97.424653,58.688364,99.195454,0.190631,109.336323,109.336323,649.0,...,84.853018,3.191185e+05,122.341734,121.163053,104.833596,113.813513,-0.203451,-0.118756,97.134776,33.477185
4,45.288313,-138.038339,-1.226468e+06,73.602326,52.136797,107.304441,-1.013413,58.622933,58.622933,614.0,...,105.163054,-1.769839e+06,135.023802,133.033379,90.345308,120.139598,-0.241381,-0.864557,94.354414,133.792957


In [26]:
automatic_scoring_cv(X, y)

'cv + roc_auc: 0.8605256098680603'

In [27]:
automatic_scoring_tt(X, y)

'tt + f1_score: 0.7928756104567654'

In [28]:
X['target'] = y
X.to_csv('tmp/selected_data.csv')

### 4. Feature ranking with recursive feature elimination and cross-validated selection of the best number of features.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('tmp/selected_data_2.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

df.head()

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

sel = RFECV(SVC(kernel="linear"), step=1, cv=5, n_jobs=-1)
sel = sel.fit(X, y)
columns = np.asarray(X.columns.values)
support = np.asarray(sel.get_support())
columns_with_support = columns[support]

In [ ]:
X = X[columns_with_support]
X['target'] = y
X.to_csv('tmp/selected_data_3.csv')

In [ ]:
automatic_scoring_cv(X, y)